# Mapping

For mapping trained models are used. Image segmentation is performed after the same prepatation, as was made for data for algorithm training:
1) getting additional information from bands -- spectral indicies \
2) data normalisation

In [1]:
#libraries

import os
import pandas as pd
import numpy as np
import rasterio as rio
from itertools import product
from rasterio import windows
import matplotlib.pyplot as plt
from sklearn import preprocessing

from osgeo import gdal
from osgeo.gdalconst import GDT_Int16

import joblib

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#helpers to get classified data
from utils import col_names, col_names_full, get_spectral_indices, to_2d_array, save_tif, get_dataset, simple_classifier, get_raster

# Models read

In [4]:
svc_model = joblib.load('models/svc_best_model.joblib')
rf_model = joblib.load('models/rf_best_model.joblib')
knn_model = joblib.load('models/knn_best_model.joblib')

# Raster classification

In [7]:
path = 'data/2021_kola_median_composite.tif'

get_raster(path=path,
           model=knn_model, output_name='data/knn_best_model_2021')

Done


# Conclusion

Visual inspection of results disproove the highest detection accuracy of SMOTE: it positevely detects areas do not belond to the urban class, such as one of the hills within the ROI -- Nyuduayvenchor, did not detect urban classes at the center of the image, negatively biased to flooded vegetation.
The most promissing algorithm according to the visual inspection of results is Random Forest. However, on the contrary, it is positevely biased to urban class, while the bare soils could be mapped in the left bottom corner. Such misclassification could be a result of mixing up the disturbed areas with urban classes, which could be further improved by the class balancing and label data enrichment.